In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report

import numpy as np
import pandas as pd
import os
import io
import time

## prepare the dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')
DATA_DIR = '/content/drive/My Drive/DLDATA/Crop/'

Mounted at /content/drive


In [16]:
class AttentionModel(tf.keras.Model):
    def __init__(self, batch_size, input_dim, hidden_dim, output_dim, recurrent_layers, dropout_p):
        super(AttentionModel, self).__init__()

        self.batch_size = batch_size
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim
        self.recurrent_layers = recurrent_layers
        self.dropout_p = dropout_p

        self.input_embeded = tf.keras.layers.Dense(
            hidden_dim//2, activation='tanh')

        self.dropout = tf.keras.layers.Dropout(dropout_p)

        self.rnn_layers = []
        for _ in range(0, recurrent_layers):

            rnn_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(hidden_dim,
                                                                           return_sequences=True,
                                                                           return_state=True))
            self.rnn_layers.append(rnn_layer)

        self.self_attention = tf.keras.Sequential([
            tf.keras.layers.Dense(hidden_dim*2, activation='relu'),
            tf.keras.layers.Dense(1)
        ])

        self.scale = 1.0/np.sqrt(hidden_dim)

        self.output_linear = tf.keras.layers.Dense(self.hidden_dim)
        self.label = tf.keras.layers.Dense(output_dim)

    def call(self, input_sentences, batch_size=None):

        input = self.input_embeded(input_sentences)
        input = self.dropout(input)

        for i, _ in enumerate(self.rnn_layers):
            output, forward_h, forward_c, backward_h, backward_c = self.rnn_layers[i](
                input)

        attn_ene = self.self_attention(output)

        # scale
        attn_ene = tf.math.scalar_mul(self.scale, attn_ene)
        attns = tf.nn.softmax(attn_ene, axis=1)

        context_vector = attns * output
        final_inputs = tf.reduce_sum(context_vector, axis=1)
        final_inputs2 = tf.reduce_sum(output, axis=1)

        combined_inputs = tf.concat([final_inputs, final_inputs2], axis=1)

        logits = self.label(combined_inputs)

        return logits

In [17]:
# Filtered data
data_path = '/content/drive/My Drive/DLDATA/Crop/cleaned_data_25753.csv'
df_all = pd.read_csv(data_path)

# pick up only NDVI,and paddocktyp
labels = df_all.iloc[:,2]
df_all = df_all.iloc[:, 12:].copy()
mask = (df_all <= 1).all(axis=1)
df_all = df_all[mask]
labels = labels[mask]
X = df_all
y = labels


le = LabelEncoder()
le.fit(y)
print(le.classes_)
class_names = le.classes_
y = le.transform(y)


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=15, stratify=y)


X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

print('X_train:{}'.format(X_train.shape))
print('X_test:{}'.format(X_test.shape))
print('y_train:{}'.format(y_train.shape))
print('y_test:{}'.format(y_test.shape))

# balance data
# ros = RandomOverSampler(random_state=SEED)
# X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

X_train_resampled, y_train_resampled = X_train, y_train

# check sample no.
# unique_elements, counts_elements = np.unique(y_train, return_counts=True)
# print("Frequency of unique values of the said array:")
# print(np.asarray((unique_elements, counts_elements)))



# DataLoader definition
# model hyperparameters
INPUT_DIM = 1
OUTPUT_DIM = 5
HID_DIM = 64
DROPOUT = 0.1
RECURRENT_Layers = 2
# LR = 0.001  # learning rate
EPOCHS = 400
BATCH_SIZE = 32
num_classes = 5



# unique_elements, counts_elements = np.unique(y_train, return_counts=True)
# weights = [1/i for i in counts_elements]
# weights[2] = weights[2]/15
# print(np.asarray((unique_elements, counts_elements)))
# print(weights)
# samples_weight = np.array([weights[t] for t in y_train])
# samples_weights = torch.FloatTensor(samples_weight).to(device)
# class_weights = torch.FloatTensor(weights).to(device)
# sampler = torch.utils.data.sampler.WeightedRandomSampler(samples_weights, len(X_train_resampled),replacement=True)

# prepare PyTorch Datasets


model = AttentionModel(BATCH_SIZE, INPUT_DIM, HID_DIM,
                           OUTPUT_DIM, RECURRENT_Layers, DROPOUT)


early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                patience=10,
                                                mode='min')


model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            optimizer=tf.optimizers.Adam(),
            metrics=['accuracy'])


baseline_history = model.fit(
    X_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks = [early_stopping],
    validation_data=(X_test, y_test))

# print(model.summary())

['Barley' 'Canola' 'Chick Pea' 'Lentils' 'Wheat']
X_train:(22851, 276, 1)
X_test:(2540, 276, 1)
y_train:(22851,)
y_test:(2540,)
Epoch 1/400
715/715 [==============================] - 19s 26ms/step - loss: 1.5076 - accuracy: 0.3912 - val_loss: 1.2168 - val_accuracy: 0.4220
Epoch 2/400
715/715 [==============================] - 18s 25ms/step - loss: 1.2797 - accuracy: 0.4048 - val_loss: 1.2288 - val_accuracy: 0.4492
Epoch 3/400
715/715 [==============================] - 18s 25ms/step - loss: 1.2306 - accuracy: 0.4162 - val_loss: 1.2063 - val_accuracy: 0.4075
Epoch 4/400
715/715 [==============================] - 18s 25ms/step - loss: 1.2175 - accuracy: 0.4222 - val_loss: 1.1972 - val_accuracy: 0.4232
Epoch 5/400
715/715 [==============================] - 18s 25ms/step - loss: 1.1986 - accuracy: 0.4253 - val_loss: 1.1960 - val_accuracy: 0.4264
Epoch 6/400
715/715 [==============================] - 18s 25ms/step - loss: 1.1887 - accuracy: 0.4259 - val_loss: 1.2117 - val_accuracy: 0.4346
Ep

In [ ]:
# checkpoint_dir = './training_checkpoints'
# checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
# checkpoint = tf.train.Checkpoint(optimizer=optimizer,
#                                  encoder=encoder,
#                                  decoder=decoder)